# Laboratorio 1 - Informe

### Grupo 4:
     - S. Calvo C.I 5.711.417-7     
     - X. Iribarnegaray C.I 5.253.705-9
     - J. Simonelli C.I 5.405.358-4

## 1. Objetivo

El objetivo de este laboratorio es:
- Implementar el algoritmo ID3, añadiendo el hiperparámetro *max_range_split*, que determina la cantidad máxima de rangos en los que se puede partir un atributo númerico.
- Utilizar scikit-learn para el preprocesamiento de datos y la creación de modelos basados en árboles de decisión.
- Evaluar y comparar los modelos generados.

## 2. Diseño
### 2.1 Algoritmo

In [16]:
def id3(dataset, target, features, continuous_features, max_range_splits, intact_dataset):
    if len(features) == 0 or len(dataset[target].value_counts().index) == 1:
        # value_counts[0] is either the only or the most common target value left in the current dataset.
        return dataset[target].value_counts().index[0] 
 
    best, best_splits = best_feature(dataset, target, features, continuous_features, max_range_splits)
    decision_tree = {best: {}}
    
    new_features = features.copy()
    new_features.remove(best)
    
    original_dataset = intact_dataset
    
    if best_splits:
        original_dataset = split_dataset(intact_dataset, best, best_splits)
        dataset = split_dataset(dataset, best, best_splits)
        
    for value in original_dataset[best].value_counts().index:
        examples = dataset.loc[dataset[best] == value]
        if (len(examples) == 0):
            decision_tree[best][value] = original_dataset.loc[original_dataset[best] == value][target].value_counts().index[0]
        else:
            decision_tree[best][value] = id3(examples, target, new_features, continuous_features, max_range_splits, intact_dataset)
    
    return decision_tree

El algoritmo ID3 es un algoritmo recursivo que, dado un dataset, construye un árbol de decisión de forma recursiva. A continunación describiremos nuestra implementación del algoritmo.

#### Casos base:
- Si no quedan features (atributos) a analizar, etiquetar la hoja con el valor más común en el dataset restante.
- Si todos los ejemplos restantes en el dataset tienen el mismo valor target, etiquetar la hoja con ese valor. 

En este caso, para ambas condiciones clasificamos la hoja con el valor `dataset[target].value_counts().index[0]`. La función `value_counts()` retorna los valores de la columna target en el dataset, ordenados de más a menos común. Al tomar el primero de estos valores, nos aseguramos de etiquetar la hoja con el más común, para el primer caso, y de etiquetar la hoja con el único valor restante en el dataset para el segundo.

#### Recursión:
Si no se cumplen las condiciones de los casos base, entonces, el siguiente paso es elegir el mejor atributo: en el caso de nuestra implementación, el atributo que maximiza la ganancia (def. en Teorico). Este se obtiene mediante `best_feature`, que retorna tanto el mejor atributo como, en el caso de que este sea continuo, el mejor o mejores puntos de corte (`best_splits`). Si el atributo es contniuo, es decir, `best_splits` está definido, discretizamos utilizando estos puntos de corte los valores del mejor atributo, tanto en el conjunto de ejemplos restantes como en una copia del dataset_original.

Luego, por cada valor posible que puede tomar el mejor atributo (en nuestro caso, los valores que toma este atributo en el dataset original o en la copia discretizada que obtuvimos en el paso anterior), construimos una rama del árbol. En estas ramas, si no hay ejemplos con el mismo valor para el atributo en cuestión, se asigna el valor objetivo más común de los ejemplos del conjunto de datos original, cuyo valor de atributo sea el mismo que el que está siendo evaluado en el momento. De lo contrario, se llama de forma recursiva a la función id3, quitando este atributo del conjunto de atributos a evaluar y usando como nuevo dataset los ejemplos del mismo valor de atributo.

In [17]:
def best_feature(dataset, target, features, continuous_features, max_range_splits):
    conditional_entropies = []
    continuous = {}
    for feature in features:
        # Continuous-Valued feature 
        if feature in continuous_features:
            aux_conditional_entropy, best_split = get_splits(dataset, feature, target, max_range_splits)
            conditional_entropies.append(aux_conditional_entropy)
            continuous[feature] = best_split
        else :
            res = 0
            for value, count in dataset[feature].value_counts().items():
                res += count*entropy(dataset.loc[dataset[feature] == value], target)
            conditional_entropies.append(res / dataset.shape[0])
    best_feature = features[conditional_entropies.index(min(conditional_entropies))]
    
    if not (best_feature in continuous):
        return best_feature, None
    return best_feature, continuous[best_feature]

La función best_attribute se encarga de calcular, dado un conjunto de ejemplos, el atributo que maximiza la ganancia de información, como se definió en el marco teórico. De manera equivalente, también puede decirse que selecciona el atributo que minimiza el sustraendo en la fórmula de la ganancia.\
\
Si el atributo a evaluar es continuo, los posibles puntos de corte para categorizar los valores de este atributo en rangos se obtienen mediante la función `get_splits(dataset, feature, target, max_range_splits)`. Esta función realiza lo siguiente:
- Ordena de menor a mayor de valores: Ordena los valores del atributo en el conjunto de ejemplos de menor a mayor.
- Identifica los puntos de corte: Recorre la columna target en ese orden, y cuando el valor de target cambia (por ejemplo, entre las filas i e i+1), se registra el promedio entre el valor del atributo en la fila i y el valor en la fila i+1 como un posible punto de corte.\

Para reducir el tiempo de ejecución del algoritmo, si la cantidad de posibles puntos de corte supera los 50, se seleccionan 50 puntos de corte al azar. Luego, se generan todas las combinaciones posibles de estos puntos, ordenadas en tamaños de hasta max_range_split - 1.\
\
Si el número de posibles puntos de corte es menor o igual a 50, procedemos por calcular directamente el valor del sustraendo mencionado previamente.\
\
El objetivo de este proceso es dividir los valores continuos en rangos mediante puntos de corte que se calculan como el promedio de los valores del atributo para dos ejemplos consecutivos con diferentes valores en el target. Dependiendo del valor del hiperparámetro max_range_splits (que puede tomar valores de 2 o 3), utilizamos combinaciones de estos puntos para crear los rangos. Finalmente, se obtiene un atributo de valores discretos.\
\
Es importante considerar que, en datasets de gran tamaño, es probable que existan muchos posibles puntos de corte. Por ello, es necesario identificar el mejor o los mejores puntos de corte. De manera similar al cálculo del "mejor atributo", se evalúa la ganancia de información para cada posible split, seleccionando aquel que minimice el segundo factor de la fórmula de la ganancia. Para este propósito, se utiliza la función get_splits.


In [18]:
def get_splits(dataset, feature, target, max_range_splits):
    min_conditional_entropy = 2
    dataset = dataset.sort_values(by=feature)
    current_target = dataset[target].iloc[0]
    dataset_size = dataset.shape[0]
    candidate_splits = []
    best_splits = []
    
    # Finding splits
    for i in range(1, dataset_size):
        if current_target != dataset[target].iloc[i]:
            candidate_splits.append((dataset[feature].iloc[i-1] + dataset[feature].iloc[i])/2)
            current_target = dataset[target].iloc[i]
    
    sample = candidate_splits
    if len(candidate_splits) > 50:
        sample = random.sample(candidate_splits, 50)
    
    splits = generate_combinations(sample, max_range_splits)
 
    for split in splits:
        splitted_dataset = split_dataset(dataset, feature, split)
        aux_conditional_entropy = 0
        for value, count in splitted_dataset[feature].value_counts().items():
            aux_conditional_entropy += count*entropy(splitted_dataset.loc[splitted_dataset[feature] == value], target)
        aux_conditional_entropy = aux_conditional_entropy / splitted_dataset.shape[0]
            
        if (aux_conditional_entropy < min_conditional_entropy):
            min_conditional_entropy = aux_conditional_entropy
            best_splits = split
            
    return (min_conditional_entropy, best_splits)

In [19]:
import numpy as np

def entropy(dataset, target):
    # value_counts() returns a Series containing the counts of unique values
    values = dataset[target].value_counts()
    # shape returns the size of dataset, shape[0] being the number of rows
    total = dataset.shape[0]
    p0 = values.iloc[0]/total
    if (len(values) > 1):
        p1 = values.iloc[1]/total
        return -(p0)*np.log2(p0) - (p1) * np.log2(p1)
    else: 
        return -(p0)*np.log2(p0)


### 2.3 Evaluación


A lo largo de la siguiente sección de **Experimentación**, dividiremos el dataset original en conjuntos de entrenamiento y de prueba, en diferentes proporciones para experimentar con el sobreajuste del modelo. Más allá de esto, para la prueba de nuestro algoritmo y la comparación entre los clasificadores de scikit-learn con nuestra implementación de ID3 fijaremos la división en 80/20 train-test.\
\
Para medir la correctitud de nuestra implementación, utilizaremos la métrica de *accuracy* al asignar valores objetivo a nuevos ejemplos de instancias, y la calcularemos con la siguiente función:

In [20]:
def test_instances(tree, dataset):
    res = 0
    for i in range(0,dataset.shape[0]):
        if classify_instance(tree, dataset.iloc[i]) == dataset.iloc[i][target]:
            res = res + 1 
    return (res/dataset.shape[0])*100

Esta recorrerá todos los ejemplos del dataset de prueba, y comparará su valor objetivo con el obtenido a partir del arbol de decisión calculado por ID3. El proceso de asignar un valor objetivo a un ejemplo nuevo es llevado a cabo por la función `classify_instance`:

In [21]:
def classify_instance(tree, instance):
    if isinstance(tree, dict):
        feature, branches = next(iter(tree.items()))
        feature_value = instance[feature]
        if isinstance(branches, dict):
            for condition, subtree in branches.items():
                if (isEqual(feature_value, condition)):
                    return classify_instance(subtree, instance)
        else:
            return branches
    else:
        return tree

Comparando el valor del feature por el que se está iterando con la condición descrita en la posición actual del arbol (mediante la función `isEqual`), la función recorrerá el arbol hasta lograr asignar un valor objetivo a la instancia\
\
Junto con la métrica accuracy, hemos utilizado también el tiempo de ejecución para medir la correctitud de nuestra implementación, y una vez conseguimos una accuracy de alrededor 80% y un tiempo de ejecución razonable (dependiendo del parámetro `max_range_splits`), consideramos nuestra implementación como correcta.

## 3. Experimentación

#### Comparativa entre preprocesamiento y max_range_splits = 2 con diferentes ratios train/test
En el siguiente código, ejecutaremos nuestra implementación de ID3 con el hiperparámetro `max_range_splits = 2` para distintas proporciones de división del dataset entre conjunto de entrenamiento y de prueba. Esto último con el objetivo de evaluar cómo varía la precisión del modelo a medida que cambiamos la proporción de datos destinados al entrenamiento y a la prueba, lo que nos permitirá identificar posibles casos de sobreajuste cuando el modelo se ajusta demasiado a los datos de entrenamiento y no generaliza correctamente a datos nuevos.\
\
Además, para cada división conjuntos entrenamiento-prueba, se ejecutará ID3 tanto para el dataset original como para uno preprocesado, y desplegará el tiempo de ejecución para ambos casos.\
\
Con respecto al preprocesamiento mencionado, lo que realizamos para efectivamente preprocesar el dataset es discretizar todos los atributos continuos previo a la ejecución del algoritmo, para así evitar la discretización en tiempo de ejecución.

In [22]:
from id3 import get_splits, split_dataset, id3, split_into_train_test, test_instances, init
from datetime import datetime
import random

dataset, features, continuous_features, target = init()

for i in range(50,100,10):
       print('\n',i,'% entrenamiento, ',100-i, '% test' )
       train_ds, test_ds = split_into_train_test(dataset,i/100)
       
       # Preprocessed
       preprocessed_dataset = train_ds.copy()
       for cont_feature in continuous_features:
              entropy, splits = get_splits(preprocessed_dataset,cont_feature,target,2)
              preprocessed_dataset = split_dataset(preprocessed_dataset,cont_feature,splits)
       startTime = datetime.now()
       preprocessed_decision_tree = id3(preprocessed_dataset, target, features, [], 2, preprocessed_dataset)
       preprocessed_time = datetime.now() - startTime
       acierto_pre = test_instances(preprocessed_decision_tree,test_ds)
       print('Preprocessed: ', end='')
       print('Accuracy: ',acierto_pre,'%', ' Time: ', preprocessed_time)
       
       
       # Max_Range_Splits_2
       startTime = datetime.now()
       max_range_split_2_decision_tree = id3(train_ds,target,features, continuous_features, 2, train_ds)
       max_range_split_2_time = datetime.now() - startTime
       acierto_run = test_instances(max_range_split_2_decision_tree,test_ds)
       print('Max Range Split 2: ', end='')
       print('Accuracy: ',acierto_run,'%', ' Time: ', max_range_split_2_time)



 50 % entrenamiento,  50 % test
Preprocessed: Accuracy:  82.89719626168224 %  Time:  0:00:05.187340
Max Range Split 2: Accuracy:  82.14953271028037 %  Time:  0:00:19.916350

 60 % entrenamiento,  40 % test
Preprocessed: Accuracy:  82.12616822429906 %  Time:  0:00:05.771565
Max Range Split 2: Accuracy:  82.00934579439252 %  Time:  0:00:19.757810

 70 % entrenamiento,  30 % test
Preprocessed: Accuracy:  81.30841121495327 %  Time:  0:00:06.654187
Max Range Split 2: Accuracy:  83.02180685358256 %  Time:  0:00:23.023650

 80 % entrenamiento,  20 % test
Preprocessed: Accuracy:  80.14018691588785 %  Time:  0:00:07.987066
Max Range Split 2: Accuracy:  79.67289719626169 %  Time:  0:00:24.466648

 90 % entrenamiento,  10 % test
Preprocessed: Accuracy:  79.90654205607477 %  Time:  0:00:08.379983
Max Range Split 2: Accuracy:  78.03738317757009 %  Time:  0:00:31.400917


#### Comparativa entre preprocesamiento y max_range_splits = 3 con diferentes ratios train/test
El objetivo de esta comparativa es análoga a la anterior, solo que cambiando el valor de `max_range_splits` a 3. Por esto último, el tiempo de ejecución de todo el código aumenta drásticamente.

In [24]:
from id3 import get_splits, split_dataset, id3, split_into_train_test, test_instances, init
from datetime import datetime
import random

dataset, features, continuous_features, target = init()

for i in range(50,100,10):
       print('\n',i,'% entrenamiento, ',100-i, '% test' )
       train_ds, test_ds = split_into_train_test(dataset,i/100)
       
       # Preprocessed
       preprocessed_dataset = train_ds.copy()
       for cont_feature in continuous_features:
              entropy, splits = get_splits(preprocessed_dataset,cont_feature,target,2)
              preprocessed_dataset = split_dataset(preprocessed_dataset,cont_feature,splits)
       startTime = datetime.now()
       preprocessed_decision_tree = id3(preprocessed_dataset, target, features, [], 2, preprocessed_dataset)
       preprocessed_time = datetime.now() - startTime
       acierto_pre = test_instances(preprocessed_decision_tree,test_ds)
       print('Preprocessed: ', end='')
       print('Accuracy: ',acierto_pre,'%', ' Time: ', preprocessed_time)
       
       # Max_Range_Splits_3
       startTime = datetime.now()
       max_range_split_3_decision_tree = id3(train_ds,target,features, continuous_features, 3, train_ds)
       max_range_split_3_time = datetime.now() - startTime
       acierto_run = test_instances(max_range_split_3_decision_tree,test_ds)
       print('Max Range Split 3: ', end='')
       print('Accuracy: ',acierto_run,'%', ' Time: ', max_range_split_3_time)


 50 % entrenamiento,  50 % test
Preprocessed: Accuracy:  81.77570093457945 %  Time:  0:00:05.543877
Max Range Split 3: Accuracy:  84.39252336448598 %  Time:  0:03:08.861896

 60 % entrenamiento,  40 % test
Preprocessed: Accuracy:  83.41121495327103 %  Time:  0:00:05.461840
Max Range Split 3: Accuracy:  80.8411214953271 %  Time:  0:04:23.020943

 70 % entrenamiento,  30 % test
Preprocessed: Accuracy:  78.19314641744548 %  Time:  0:00:07.355429
Max Range Split 3: Accuracy:  81.15264797507788 %  Time:  0:04:35.594333

 80 % entrenamiento,  20 % test
Preprocessed: Accuracy:  77.57009345794393 %  Time:  0:00:08.072927
Max Range Split 3: Accuracy:  78.97196261682244 %  Time:  0:05:01.331624

 90 % entrenamiento,  10 % test
Preprocessed: Accuracy:  78.03738317757009 %  Time:  0:00:08.293726
Max Range Split 3: Accuracy:  75.70093457943925 %  Time:  0:06:02.981560


#### Random Forest vs Decision Tree Classifier vs Max_range_splits = 2 vs Max_range_splits = 3
Finalmente, ejecutamos los algoritmos Decision Tree Classifier y Random Forest de la librería scikit-learn sobre el conjunto de datos, y comparamos su accuracy con nuestra implementación de ID3.\
\
Brevemente, el algoritmo Random Forest se basa en generar subconjuntos a partir del conjunto original de datos, y para cada uno de estos se calcula un árbol de decisión independiente. Una vez calculados, se promedia los resultados obtenidos para llegar a una mejor predicción final.\
Luego, Decision Tree Classifier es el cálculo de árboles de decisión implementado por scikit-learn.\
\
Cabe destacar que fue necesario preprocesar el dataset para la utilización de estos dos nuevos algoritmos, ya que estos están especificados para datasets de variables continuas.\
Para esto, utilizamos One Hot Encoding, cuya función es la de para cada atributo categórico, crear una columna nueva por cada posible valor que este atributo pueda tomar. De esta manera, elimina los atributos categóricos, y nos permite ejecutar los nuevos algoritmos sobre el nuevo conjunto de datos.\
Observación: Como la gran mayoría de atributos categóricos tienen posible valor 0 o 1, solo tomamos el atributo `trt` para ejecutar One Hot Encoding.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from id3 import init, split_into_train_test, id3, test_instances

dataset, features, continuous_features, target = init()

X = dataset.drop(target, axis=1)
y = dataset[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

ohe = OneHotEncoder(sparse_output=False)

discrete_features = ['trt']

for feat in discrete_features:
    ohe.fit(dataset[feat].to_numpy().reshape(-1, 1))
    
    # Transform the training and test data using the fitted OneHotEncoder
    # Converts the categorical feature in X_train and X_test to one-hot encoded format
    new_train = ohe.transform(X_train[feat].to_numpy().reshape(-1,1))
    new_test = ohe.transform(X_test[feat].to_numpy().reshape(-1,1))
    
    # Create column names for the new one-hot encoded features
    column_names = [f"{feat}_{cat}" for cat in ohe.categories_[0]]
    
    for i, col_name in enumerate(column_names):
        # Add the new one-hot encoded columns to the X_train and X_test DataFrame
        X_train[col_name] = new_train[:, i]
        X_test[col_name] = new_test[:, i]
    
    X_train.drop(feat, axis=1)
    X_test.drop(feat, axis=1)

    

random_forest = RandomForestClassifier(random_state=0)
random_forest.fit(X_train, y_train)

decision_tree_classifier = DecisionTreeClassifier(random_state=0)
decision_tree_classifier.fit(X_train, y_train)

y_pred_random_forest = random_forest.predict(X_test)
accuracy_random_forest = accuracy_score(y_test, y_pred_random_forest)
print('Random forest accuracy: ', accuracy_random_forest*100, '%')

y_pred_decission_tree = decision_tree_classifier.predict(X_test)
accuracy_decision_tree_classifier = accuracy_score(y_test, y_pred_decission_tree)
print('Decission Tree Classifier accuracy: ', accuracy_decision_tree_classifier*100, '%')

train_ds, test_ds = split_into_train_test(dataset,0.8)

max_range_split_2_decision_tree = id3(train_ds,target,features, continuous_features, 2, train_ds)
acierto_split_2 = test_instances(max_range_split_2_decision_tree,test_ds)
print('Max Range Split 2 accuracy: ',acierto_split_2,'%')

max_range_split_3_decision_tree = id3(train_ds,target,features, continuous_features, 3, train_ds)
acierto_split_3 = test_instances(max_range_split_3_decision_tree,test_ds)
print('Max Range Split 3 accuracy: ',acierto_split_3,'%')




Random forest accuracy:  89.01869158878505 %
Decission Tree Classifier accuracy:  83.41121495327103 %
Max Range Split 2 accuracy:  78.50467289719626 %
Max Range Split 3 accuracy:  83.41121495327103 %


## 4. Conclusión

En primer lugar, hemos observado que a medida que se utilzia una mayor porción del conjunto de datos original para el entrenamiento del modelo, disminuye más la accuracy del modelo. Esto es consecuencia del overfitting, que a medida que el modelo es expuesto a una mayor cantidad de datos de entrenamiento, este se sobreajusta a estos datos y se vuelve cada vez peor a la hora de generalizar.\
Esta tendencia la observamos para ambas variaciones del parámetro `max_range_splits`.\
\
No hemos observado una clara tendencia de mayor accuracy en una variación del hiperparámetro sobre otra. Únicamente observamos un mayor tiempo de ejecución para la variación `max_range_splits=3`. Podemos realizar una observación análoga para las instancias que discretizamos el dataset previo a la ejecución del algoritmo, donde la variación más notable fue el tiempo de ejecución\
\
Por último, con respecto a los algoritmos `RandomForest` y `DecisionTreeClassifier`, hemos observado que tienen una mejor accuracy que nuestras implementaciones, y drásticamente menor tiempo de ejecución.